Step 1: Data the dataset


In [ ]:
import pandas as pd



In [ ]:
# Load datasets
orders = pd.read_csv('/content/drive/MyDrive/MLdev/dispatchbrain/events.csv')
events = pd.read_csv('/content/drive/MyDrive/MLdev/dispatchbrain/orders.csv')
weather = pd.read_csv('/content/drive/MyDrive/MLdev/dispatchbrain/weather.csv')


In [ ]:
orders.head()

,order_id,event_type,event_ts
0,O1,created,2025-07-01 11:14:00
1,O1,assigned,2025-07-01 11:16:00
2,O1,picked,2025-07-01 11:25:00
3,O1,delivered,2025-07-01 11:35:00
4,O2,created,2025-07-01 15:02:00


In [ ]:
events.head()

,order_id,city,service_type,pickup_lat,pickup_lon,drop_lat,drop_lon,created_at,promised_at,delivered_at,distance_km,eta_minutes,is_late
0,O1,KHI,grocery,24.80302,67.02307,24.848467,67.051411,2025-07-01 11:14:00,2025-07-01 11:44:00,2025-07-01 11:35:00,10.28,21.5,0
1,O2,ISB,grocery,33.74317,73.09969,33.781976,73.121179,2025-07-01 15:02:00,2025-07-01 16:02:00,2025-07-01 15:09:00,1.69,7.1,0
2,O3,KHI,grocery,24.83310,67.02072,24.877464,67.050722,2025-07-01 19:36:00,2025-07-01 20:21:00,2025-07-01 19:49:00,4.35,13.9,0
3,O4,ISB,grocery,33.72600,73.03126,33.752427,73.049648,2025-07-01 15:38:00,2025-07-01 16:08:00,2025-07-01 15:43:00,2.17,-0.1,0
4,O5,LHR,food,31.47278,74.28286,31.484959,74.300158,2025-07-01 16:12:00,2025-07-01 16:52:00,2025-07-01 16:21:00,6.99,9.0,0


In [ ]:
weather.head()

,city,ts,temp_c,rain_mm,wind_kmph
0,KHI,2025-07-01 00:00:00,34.0,0.00,12
1,KHI,2025-07-01 01:00:00,33.9,0.39,11
2,KHI,2025-07-01 02:00:00,37.5,0.00,4
3,KHI,2025-07-01 03:00:00,31.6,0.00,6
4,KHI,2025-07-01 04:00:00,28.3,0.00,5


In [ ]:
orders.shape

(24000, 3)

In [ ]:
events.shape

(6000, 13)

In [ ]:
weather.shape

(2160, 5)

In [ ]:
orders.columns

Index(['order_id', 'event_type', 'event_ts'], dtype='object')

In [ ]:
weather.columns

Index(['city', 'ts', 'temp_c', 'rain_mm', 'wind_kmph'], dtype='object')

In [ ]:
orders.columns

Index(['order_id', 'event_type', 'event_ts'], dtype='object')

**Convert the timestamps to datetime format**

In [ ]:
orders['event_ts'] = pd.to_datetime(orders['event_ts'], errors='coerce')
events['created_at'] = pd.to_datetime(events['created_at'], errors='coerce')
events['promised_at'] = pd.to_datetime(events['promised_at'], errors='coerce')
events['delivered_at'] = pd.to_datetime(events['delivered_at'], errors='coerce')
weather['ts'] = pd.to_datetime(weather['ts'], errors='coerce')

**Pivot the orders file so that each event type becomes a separate column for every order**

In [ ]:
orders_pivot = orders.pivot(index='order_id', columns='event_type', values='event_ts').reset_index()

**Merge the pivoted orders with the events file**

In [ ]:
orders_merged = pd.merge(orders_pivot, events, on='order_id', how='left')

**Extract the pickup hour so it can be matched with the weather data.**

In [ ]:
orders_merged['pickup_hour'] = orders_merged['created_at'].dt.floor('H')

/tmp/ipython-input-2429747417.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  orders_merged['pickup_hour'] = orders_merged['created_at'].dt.floor('H')


**Merge the data with the weather information.**

In [ ]:
orders_final = pd.merge(
    orders_merged,
    weather,
    left_on=['city', 'pickup_hour'],
    right_on=['city', 'ts'],
    how='left'
)

**Drop the extra column and fill in the missing weather values**

In [ ]:
orders_final.drop(columns=['ts'], inplace=True)
orders_final[['temp_c', 'rain_mm', 'wind_kmph']] = orders_final[['temp_c', 'rain_mm', 'wind_kmph']].fillna(0)

**Save the final cleaned file.**

In [ ]:
import os
os.makedirs('data/processed', exist_ok=True)

In [ ]:
# Step 2: Save it as a proper CSV file
orders_final.to_csv("data/processed/merged_orders_cleaned.csv", index=False)



In [ ]:
orders_final.head()

,order_id,assigned,created,delivered,picked,city,service_type,pickup_lat,pickup_lon,drop_lat,...,created_at,promised_at,delivered_at,distance_km,eta_minutes,is_late,pickup_hour,temp_c,rain_mm,wind_kmph
0,O1,2025-07-01 11:16:00,2025-07-01 11:14:00,2025-07-01 11:35:00,2025-07-01 11:25:00,KHI,grocery,24.80302,67.02307,24.848467,...,2025-07-01 11:14:00,2025-07-01 11:44:00,2025-07-01 11:35:00,10.28,21.5,0,2025-07-01 11:00:00,32.9,0.00,13
1,O10,2025-07-01 21:35:00,2025-07-01 21:33:00,2025-07-01 21:38:00,2025-07-01 21:42:00,LHR,food,31.45807,74.31694,31.477321,...,2025-07-01 21:33:00,2025-07-01 22:03:00,2025-07-01 21:38:00,1.89,1.6,0,2025-07-01 21:00:00,24.2,0.00,8
2,O100,2025-07-01 21:24:00,2025-07-01 21:21:00,2025-07-01 21:32:00,2025-07-01 21:32:00,KHI,grocery,24.92562,67.04845,24.971180,...,2025-07-01 21:21:00,2025-07-01 22:01:00,2025-07-01 21:32:00,4.73,11.8,0,2025-07-01 21:00:00,33.6,0.00,5
3,O1000,2025-07-05 17:07:00,2025-07-05 17:05:00,2025-07-05 17:24:00,2025-07-05 17:13:00,ISB,parcel,33.65855,73.02166,33.702327,...,2025-07-05 17:05:00,2025-07-05 17:35:00,2025-07-05 17:24:00,7.81,19.3,0,2025-07-05 17:00:00,28.7,0.63,9
4,O1001,2025-07-06 09:13:00,2025-07-06 09:09:00,2025-07-06 09:14:00,2025-07-06 09:20:00,KHI,grocery,24.91423,67.10443,24.933787,...,2025-07-06 09:09:00,2025-07-06 09:54:00,2025-07-06 09:14:00,1.08,2.3,0,2025-07-06 09:00:00,27.3,0.00,8
